In [24]:
import matplotlib.pyplot as plt
import requests
import json
from scipy import stats
import pandas as pd
from config import api_key
from pprint import pprint

In [33]:
master_dict = {
    'UNH':{
        'Dates':[],
        'Revenue':[],
        'Net Income':[],
        'Expenses':[]},
    'HUM':{
        'Dates':[],
        'Revenue':[],
        'Net Income':[],
        'Expenses':[]},
    'ANTM':{
        'Dates':[],
        'Revenue':[],
        'Net Income':[],
        'Expenses':[]},
    'AET':{
        'Dates':[],
        'Revenue':[],
        'Net Income':[],
        'Expenses':[]},
    'PRU':{
        'Dates':[],
        'Revenue':[],
        'Net Income':[],
        'Expenses':[]},
    
    }

companies = ['UNH','HUM','ANTM','AET','PRU']

for company in companies:
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{company}\
?period=quarter&limit=5&apikey={api_key}"
    try:
        response = requests.get(url).json()
    except:
        print ("Sorry, we don't have this ticker: " + company)
    for i in range(0,5):
            master_dict[company]['Dates'].append(response[i]['date'])
            master_dict[company]['Revenue'].append(response[i]['revenue'])
            master_dict[company]['Net Income'].append(response[i]['netIncome'])
            master_dict[company]['Expenses'].append(float(response[i]['revenue']) - float(response[i]['netIncome']))

pprint (master_dict)

{'AET': {'Dates': ['2018-09-30',
                   '2018-06-30',
                   '2018-03-31',
                   '2017-12-31',
                   '2017-09-30'],
         'Expenses': [14484000000.0,
                      14349000000.0,
                      14126000000.0,
                      14609000000.0,
                      14156000000.0],
         'Net Income': [1000000000,
                        1212000000,
                        1209000000,
                        244000000,
                        838000000],
         'Revenue': [15484000000,
                     15561000000,
                     15335000000,
                     14853000000,
                     14994000000]},
 'ANTM': {'Dates': ['2020-09-30',
                    '2020-06-30',
                    '2020-03-31',
                    '2019-12-31',
                    '2019-09-30'],
          'Expenses': [30936000000.0,
                       26988000000.0,
                       28098000000.0,
            

In [3]:
x = weather_data['temp']
y = weather_data['lat']
scatter = plt.scatter(x,y)
slope, intercept, rvalue, pvalue, stderr = stats.linregress(weather_data['temp'],weather_data['lat'])
regress_values = weather_data['temp'] * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
scatter = plt.plot(weather_data['temp'],regress_values,"r-")
scatter = plt.annotate(line_eq,(15,45),fontsize=15,color="red")
plt.xlabel("temp")
plt.ylabel("lat")

NameError: name 'weather_data' is not defined